
Combination
======================

### A folded domain featuring an unconformity and a fault. We start by importing


In [ ]:
# the necessary dependencies:
# Importing GemPy
import sys, os

In [ ]:
#Run this cell only if you want to run gempy in cuda mode (requires cuda and a dedicated cpu).
#Initial compilation will take a bit longer but the model will run much faster 
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cuda"

In [ ]:
import gempy as gp
import pandas as pd

### Creating the model by importing the input data and displaying it:


In [ ]:
# %%
from sandbox import _test_data
#data_path = 'https://raw.githubusercontent.com/cgre-aachen/gempy_data/master/'
#path_to_data = data_path + "/data/input_data/jan_models/"
path_to_data = _test_data.get('gempy_example_data')

geo_data = gp.create_data('combination',
                          extent=[0, 2500, 0, 1000, 100, 1200],
                          resolution=[50, 50, 50],
                          path_o=path_to_data + "model7_orientations.csv",
                          path_i=path_to_data + "model7_surface_points.csv")

# %%
geo_data.get_data()

In [ ]:
# Setting and ordering the units and series:
gp.map_stack_to_surfaces(geo_data, {"Fault_Series": ('fault'), "Strat_Series1": ('rock3'),
                                     "Strat_Series2": ('rock2','rock1'),
                                     "Basement_Series":('basement')})

geo_data.set_is_fault(['Fault_Series'])

# Plot Data
gp.plot_2d(geo_data, direction='y')

### Calculating the model:


In [ ]:
interp_data = gp.set_interpolator(geo_data, theano_optimizer='fast_run')

sol = gp.compute_model(geo_data)

In [ ]:
%%timeit
sol = gp.compute_model(geo_data)

### Displaying the result:
<div class="alert alert-warning">

**Warning:** The sandbox thread is faster when smaller resolution is used, but the cross-sections can look ugly or not understandable at all

</div>



In [ ]:
 #x direction:
gp.plot_2d(geo_data, cell_number=5,
           direction='y', show_data=False, show_boundaries=True)

# y direction:
gp.plot_2d(geo_data, cell_number=5,
           direction='x', show_data=True)

In [ ]:
# 3d view
gp.plot_3d(geo_data, image=False, show_topography=True, plotter_type='basic', notebook=True, show_lith=True)

## Initialize Sensor and Projector

In [ ]:
#save the paths to the calibration files 
from sandbox import _calibration_dir
_calibprojector = _calibration_dir + "my_projector_calibration.json"
_calibsensor = _calibration_dir + "my_sensor_calibration.json"

In [ ]:
#Import Sensor
from sandbox.sensor import Sensor
sensor = Sensor(calibsensor=_calibsensor, name="dummy")

In [ ]:
# Import projector
from sandbox.projector import Projector
projector = Projector(calibprojector=_calibprojector)

In [ ]:
#Initialize the aruco detection
from sandbox.markers import MarkerDetection
aruco = MarkerDetection(sensor=sensor)

In [ ]:
from sandbox.main_thread import MainThread
main = MainThread(sensor=sensor, projector=projector, aruco=aruco)

In [ ]:
# Start the thread 
main.run()

In [ ]:
main.widget_plot_module()

## Initialize the gempy module 

In [ ]:
from sandbox.modules.gempy import GemPyModule
gpsb= GemPyModule(geo_model = geo_data, 
                  extent = sensor.extent, 
                  box=sensor.physical_dimensions, 
                  load_examples = False,
                 )

In [ ]:
main.add_module(name='gempy', module=gpsb)

In [ ]:
#Turn on Hill Shading
gpsb.show_hillshades = True
gpsb.show_lith = True
gpsb.show_boundary = True
gpsb.show_hillshades = False
gpsb.show_contour = False
gpsb.show_only_faults = False
gpsb.show_fill_contour = False

In [ ]:




main.stop()